In [1]:
import numpy as np

In [2]:
def simplex_2nd_stage(x, A, b, c, j_b):
    m, n = A.shape
    while True:
        A_b, c_b = A[: , j_b], c[j_b]
        B = np.linalg.inv(A_b)
        u = c_b.dot(B)# calc capacity vector
        e = u.dot(A) - c.T# calc estimates

        if np.all(e[j_b == False] >= 0):
            return x, j_b

        j_0 = np.argwhere((e < 0) & (j_b == False)).min()

        z = B.dot(A[: , j_0])

        if np.all(z[: m] <= 0):
            raise Exception('There is no solution.')

        theta = np.array([x[j] / z[i]
            if z[i] > 0
            else np.infty
            for i, j in zip(range(m), np.argwhere(j_b))
        ])

        theta_0, s = theta.min(), theta.argmin()
        j_s = np.argwhere(j_b).ravel()[s]

        # update plan
        mask = (j_b == False)
        mask[j_0] = False
        x[mask] = 0
        x[j_0] = theta_0
        x[j_b] = x[j_b] - theta_0 * z

        # update basis
        j_b[j_s] = False
        j_b[j_0] = True

In [3]:
def simplex_1st_stage(A, b, c):
    m, n = A.shape
    neg_idxs = np.argwhere(b < 0).ravel()
    A[neg_idxs] *= -1
    b[neg_idxs] *= -1
    
    A_hat = np.hstack((A, np.identity(m)))
    x_0_hat = np.hstack((np.zeros(n), b))
    c_hat = np.hstack((np.zeros(n), [-1]*m))
    j_b_hat = np.hstack(([False] * n, [True] * m))
    
    x_s, j_b_s = simplex_2nd_stage(x_0_hat, A_hat, b, c_hat, j_b_hat)

    if np.all(x_s[n:] != 0):
        raise Exception('There is no solution.')

    while True:
        j_b_a = np.argwhere(j_b_s[n:])
        if not j_b_a:
            break
        k = j_b_a.max()
        A_b_s = A_hat[:,j_b_s]
        A_b_s_inv = np.linalg.inv(A_b_s)
        alpha = [A_b_s_inv[k].dot(A_hat[:,j]) for j in range(n)]
        j_0 = None
        for i in range(n):
            if i not in np.argwhere(j_b_s) and alpha[i] != 0:
                j_0 = i
                break
        if j_0 is None:
            A_hat = np.delete(A_hat, k-n, 0)
            A = np.delete(A, k-n, 0)
            b = np.delete(b, k-n)
            x_s = np.delete(x, k)
            j_b_s = np.delete(j_b_s, k)
            continue
        else:
            j_b_s[k] = False
            j_b_s[j_0] = True
    return simplex_2nd_stage(x[:n], A, b, c, j_b_s[:n])

In [4]:
A = np.array([
    [1, 3, 1],
    [0, 2, 1]
], dtype = np.float)


c = np.array([1, 2, 0], dtype = np.float)
b = np.array([4, 2], dtype = np.float)
x = np.array([7, -1, 0], dtype = np.float)
j_b = np.array([True, True, False])



simplex_1st_stage(A, b, c)

(array([ 7., -1.,  0.]), array([ True,  True, False], dtype=bool))

In [5]:
simplex_2nd_stage(x, A, b, c, j_b)

(array([ 7., -1.,  0.]), array([ True,  True, False], dtype=bool))